**Connect google drive**

In [ ]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Thu Jan  4 07:39:26 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
'Colab Notebooks'  'Getting started.pdf'   yolov3


**1) Clone, configure & compile Darknet**



In [ ]:
# Clone
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15833, done.
remote: Total 15833 (delta 0), reused 0 (delta 0), pack-reused 15833
Receiving objects: 100% (15833/15833), 14.39 MiB | 19.25 MiB/s, done.
Resolving deltas: 100% (10666/10666), done.


In [ ]:
# Configure
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile

/content/darknet


In [ ]:
# Compile
!make

mkdir -p ./obj/
mkdir -p backup
mkdir -p results
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -rdynamic -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:945:23: warning: variable ‘rgb’ set but not used []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wunused-but-set-variable-Wunused-but-set-variable]8;;]
  945 |                 float rgb[3];
      |                       ^~~
./src/image_opencv.cpp: In function ‘void cv_draw_object(image, float*, int, int, int*, float*, int*, int, char**)’:
./src/image_opencv.cpp:1443:14: warning: unused variable ‘bu

**2) Configure yolov3.cfg file**

In [ ]:
# Make a copy of yolov3.cfg
!cp cfg/yolov3-tiny.cfg cfg/yolov3_new_tiny.cfg

In [ ]:
# Change lines in yolov3.cfg file
!sed -i 's/batch=1/batch=64/' cfg/yolov3_new_tiny.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_new_tiny.cfg
!sed -i 's/max_batches = 500200/max_batches = 6000/' cfg/yolov3_new_tiny.cfg
!sed -i '135 s@classes=80@classes=2@' cfg/yolov3_new_tiny.cfg
!sed -i '177 s@classes=80@classes=2@' cfg/yolov3_new_tiny.cfg
!sed -i '127 s@filters=255@filters=21@' cfg/yolov3_new_tiny.cfg
!sed -i '171 s@filters=255@filters=21@' cfg/yolov3_new_tiny.cfg

**3) Create .names and .data files**

In [ ]:
!echo -e 'PCB\nCopper' > data/obj.names
!echo -e 'classes= 2\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data

**4) Save yolov3_training.cfg and obj.names files in Google drive**

In [ ]:
!cp cfg/yolov3_new_tiny.cfg /mydrive/yolov3/yolov3_new_testing.cfg
!cp data/obj.names /mydrive/yolov3/classes.txt

**5) Create a folder and unzip image dataset**

In [ ]:
!mkdir data/obj
!unzip /mydrive/yolov3/PCBD.zip -d data/obj

Archive:  /mydrive/yolov3/PCBD.zip
 extracting: data/obj/PCBD/classes.txt  
  inflating: data/obj/PCBD/object1.jpg  
  inflating: data/obj/PCBD/object1.txt  
  inflating: data/obj/PCBD/object10.jpg  
  inflating: data/obj/PCBD/object10.txt  
  inflating: data/obj/PCBD/object100.jpg  
  inflating: data/obj/PCBD/object100.txt  
  inflating: data/obj/PCBD/object101.jpg  
  inflating: data/obj/PCBD/object101.txt  
  inflating: data/obj/PCBD/object102.jpg  
  inflating: data/obj/PCBD/object102.txt  
  inflating: data/obj/PCBD/object103.jpg  
  inflating: data/obj/PCBD/object103.txt  
  inflating: data/obj/PCBD/object104.jpg  
  inflating: data/obj/PCBD/object104.txt  
  inflating: data/obj/PCBD/object105.jpg  
  inflating: data/obj/PCBD/object105.txt  
  inflating: data/obj/PCBD/object106.jpg  
  inflating: data/obj/PCBD/object106.txt  
  inflating: data/obj/PCBD/object107.jpg  
  inflating: data/obj/PCBD/object107.txt  
  inflating: data/obj/PCBD/object108.jpg  
  inflating: data/obj/PCBD/

**6) Create train.txt file**

In [ ]:
import glob
images_list = glob.glob("data/obj/PCBD/*.jpg")
with open("data/train.txt", "w") as f:
    f.write("\n".join(images_list))

In [ ]:
!wget https://pjreddie.com/media/files/yolov3-tiny.weights

--2024-01-25 07:57:18--  http://!wget/
Resolving !wget (!wget)... failed: Name or service not known.
wget: unable to resolve host address ‘!wget’
--2024-01-25 07:57:18--  https://pjreddie.com/media/files/yolov3-tiny.weights
Resolving pjreddie.com (pjreddie.com)... 162.0.215.52
Connecting to pjreddie.com (pjreddie.com)|162.0.215.52|:443... connected.
HTTP request sent, awaiting response... 

In [ ]:
!./darknet partial cfg/yolov3-tiny.cfg yolov3-tiny.weights yolov3-tiny.conv.15

**7) Download pre-trained weights for the convolutional layers file**

**8) Start training**

In [ ]:
!./darknet detector train data/obj.data cfg/yolov3_new_tiny.cfg yolov3-tiny.conv.15 -dont_show
# Uncomment below and comment above to re-start your training from last saved weights
#!./darknet detector train data/obj.data cfg/yolov3_training.cfg /mydrive/yolov3/yolov3_training_last.weights -dont_show

Streaming output truncated to the last 5000 lines.
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 23 Avg (IOU: 0.630683), count: 15, class_loss = 0.871383, iou_loss = 1.033184, total_loss = 1.904567 
 total_bbox = 1572404, rewritten_bbox = 0.621914 % 
5903/6000: loss=0.7 hours left=0.1
 5903: 0.657845, 0.866315 avg loss, 0.001000 rate, 0.996406 seconds, 377792 images, 0.066842 hours left
Loaded: 0.240859 seconds - performance bottleneck on CPU or Disk HDD/SSD
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 16 Avg (IOU: 0.862687), count: 4, class_loss = 0.004594, iou_loss = 0.049436, total_loss = 0.054030 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 23 Avg (IOU: 0.564220), count: 6, class_loss = 0.070047, iou_loss = 0.512678, total_loss = 0.582725 
 total_bbox = 1572414, rewritten_bbox = 0.621974 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 16 Avg (IOU: 0.886600), count: 5, class_loss = 0.136566, iou